## 社交关系形成的客观表现
社交关系的维持需要社交成本的投入，社交关系的形成则有很多原因，工作，兴趣，家庭等等社会结构即利益驱动和关系驱动。社交关系给我们的表象就是人与人存在联系，即投入的社交成本。我们通过探究社交关系的客观表现比如联系的频率，联系的质量（时长），联系的时机（上午打电话，与晚上打电话的肯定存在很大的不同，工作日打电话与周末打电话也肯定不同，通过观察不同年龄段在不同时机下联系的频率和质量能否探究其肯定能够），联系的方式多样性（短信，通话，GPS位置相近不相近，从而探究其社交关系形成的原因。社交成本的投入
- 联系的频率在不同年龄与性别之间的差异
- 联系质量在不同年龄与性别之间的差异
- 联系的时机在不同年龄与性别之间的差异（不同时间段，工作日周末）
- 不同时机在的联系的质量与频率之间的差异
#### 结论2：
- 图四：联系频率在不同年龄与性别之间的差异（均值，熵）
月均联系频率，月均联系平均时长
- 图五：联系时长在不同年龄与性别之间的差异（联系质量）
- 图六：联系的时机在不同年龄与性别之间的差异
- 图七：不同时机下联系的频率与质量的差异
从这四张图可以得出：
- 联系频率在于年龄关系，及性别之间在联系到额频率上存在差异
- 联系的质量在不同年龄与性别存在的差异
- 人们会怎样去选择联系的时机，那些时机联系的频率高，那些时段上联系的质量高，背后的原因是什么


## 1.数据整理

In [2]:
%matplotlib inline 
import seaborn as sns 
import pandas as pd
import numpy as np
def getStatsCallRecord(record_sample,freq='M'):
#     record_sample=call_record[call_record.custorm_id==2427853]
   
    cust_id=record_sample.index.unique()
   
    record_sample.set_index('call_time',inplace=True)
#     record_sample.sort_values('call_time',inplace=True)
    time_stats=lambda x,y:x.groupby('type')['time'].resample(freq,how=y).unstack().T
    
    month_mean_time=time_stats(record_sample,'sum').mean() ## 平均每月来电，拨出，未接通话时长
    
    month_time_cnt=time_stats(record_sample,'count').mean()  ## 平均每月来电，拨出，未接电话频率
    
    values_Of_time_stats=np.concatenate([month_mean_time.values/60.0,[month_mean_time[:2].sum()/60.0],month_time_cnt,[month_time_cnt[:2].sum()] ])
    
    columns_Of_time_stats=['MeanTimeOfAnswer','MeanTimeOfCall','MeanTimeOfMissed','SumTimeOfComm','MeanCntOfAnswer',
                   'MeanCntOfCall','MeanCntOfMissed','SumCntOfComm']
    stats_call_record=pd.DataFrame(data=values_Of_time_stats,index=columns_Of_time_stats,columns=cust_id).T
    return stats_call_record

In [83]:
call_record=pd.read_csv('data.csv',index_col=0,parse_dates=['call_time'])
top=call_record.custorm_id.value_counts().index[:3].values
call_record=call_record.set_index('custorm_id')
call_record=call_record.ix[top,:]
week_freq=['W-MON','W-TUE','W-WED','W-THU','W-FRI','W-SAT','W-SUN']
## 月：M；日历日：D；工作日：B
call_record=call_record[call_record.time!=0]
get_stats_df=lambda freq:call_record.groupby(level=0,group_keys=False).apply(lambda x:getStatsCallRecord(x,freq))  ## 平均每天，每月只需要更改采样频率即可，然后完成整合
 
## 一天的某个小时
# record_sample['hourOfDay']=[pd.to_datetime(line).hour for line in record_sample.index.values]
# record_sample.groupby('hourOfDay')['time'].count().plot()

In [99]:
Days=get_stats_df('D').add_prefix("Day_")
month=get_stats_df('M').add_prefix("Month_")
hour=get_stats_df('H').add_prefix("Hour_")
MDH=pd.concat([Days,month,hour],axis=1)
for line in  week_freq:
    MDH=pd.concat([MDH,get_stats_df(line).add_prefix(line[2:]+'_')],axis=1)

In [100]:
MDH

,Day_MeanTimeOfAnswer,Day_MeanTimeOfCall,Day_MeanTimeOfMissed,Day_SumTimeOfComm,Day_MeanCntOfAnswer,Day_MeanCntOfCall,Day_MeanCntOfMissed,Day_SumCntOfComm,Month_MeanTimeOfAnswer,Month_MeanTimeOfCall,...,SAT_MeanCntOfMissed,SAT_SumCntOfComm,SUN_MeanTimeOfAnswer,SUN_MeanTimeOfCall,SUN_MeanTimeOfMissed,SUN_SumTimeOfComm,SUN_MeanCntOfAnswer,SUN_MeanCntOfCall,SUN_MeanCntOfMissed,SUN_SumCntOfComm
custorm_id,,,,,,,,,,,,,,,,,,,,,
2427853,7.039518,6.881731,1.044259,13.921249,10.219512,10.426829,1.273292,20.646341,159.897619,153.364286,...,8.913043,135.440000,46.636806,44.731250,3.915972,91.368056,69.833333,71.250000,8.541667,141.083333
3621853,4.460977,4.314425,0.871721,8.775402,2.438559,2.027542,0.397872,4.466102,97.616667,86.796078,...,2.750000,30.550725,24.050483,21.384541,1.865789,45.435024,16.681159,13.869565,2.750000,30.550725
6028563,6.646343,3.342826,0.718056,9.989169,0.610024,0.751938,0.239303,1.361962,71.599242,42.063889,...,1.659341,9.451890,22.185681,12.942735,1.209357,35.128416,4.264957,5.263566,1.659341,9.528523


In [69]:
call_record=pd.read_csv('data.csv',index_col=0,parse_dates=['call_time'])
record_sample=call_record[call_record.custorm_id==2427853]
record_sample['hourOfDay']=[pd.to_datetime(line).hour for line in record_sample.call_time.values]


/usr/bin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


### 2.数据分析

### 2.1用户个人信息

In [135]:
cust_user_info=pd.read_csv("data/fdl_cust_user_info_chain_0228.csv",index_col=0).drop_duplicates('custorm_id').set_index('custorm_id')
cust_user_info["age"]=2016-cust_user_info.id_no.str.slice(6,10).astype("float")
##奇数为男性，偶数为女性
cust_user_info["sex"]=cust_user_info.id_no.str.slice(16,17).bfill().replace('','0').astype("float")
cust_user_info.drop(['id_no','reg_src'],axis=1,inplace=True)
### 去除年龄不在正常范围内的
cust_user_info=cust_user_info[(cust_user_info.age>15)&(cust_user_info<60)]

,reg_src,age,sex
custorm_id,,,
62340,Android,36.0,0.0
67050,Android,23.0,0.0
67920,Android,22.0,1.0
113250,iOS,20.0,1.0
118260,iOS,30.0,1.0
151200,iOS,24.0,1.0
151920,Android,32.0,1.0
171000,Android,26.0,1.0
179280,iOS,24.0,1.0


In [144]:
cust_user_info=cust_user_info[(cust_user_info.age>15)&(cust_user_info<60)]

MemoryError: 

### 2.2  用户统计信息

In [110]:
dayOfWeek=pd.read_csv('MSN/app_call_day_stats_0302.csv',index_col=0).set_index('custorm_id')
hourOfDay=pd.read_csv('MSN/app_call_hour_stats_0302.csv',index_col=0).set_index('custorm_id')
month=pd.read_csv('MSN/app_call_month_stats_0302.csv',index_col=0).set_index('custorm_id')
sample=pd.read_csv('MSN/app_call_sample.csv',index_col=0).set_index('custorm_id')

In [113]:
cust_user_info.head(3)

,id_no,reg_src,age,sex
custorm_id,,,,
62340,15030219800829QN04,Android,36.0,0.0
67050,32092119930309Q&0N,Android,23.0,0.0
67920,35042719940910ZN1E,Android,22.0,1.0


In [114]:
dayOfWeek.head(3)

,call_day,type,day_sum,day_mean,day_cnt
custorm_id,,,,,
3140118,4.0,2,2292.0,34.208955,67
3977826,1.0,1,1347.0,67.350000,20
1195448,5.0,2,3100.0,206.666667,15


### 2.3  合并数据

In [132]:
get_person_info=lambda  stats_df :stats_df.join(cust_user_info,how='left')

In [139]:
dayOfWeek=get_person_info(dayOfWeek)
hourOfDay=get_person_info(hourOfDay)
month=get_person_info(month)

In [141]:
c=dayOfWeek.groupby('call_day').apply(lambda x:x.groupby('age')['day_mean'].mean())

In [142]:
c

age,-5183.0,-3183.0,-2972.0,-1182.0,-181.0,15.0,16.0,17.0,18.0,19.0,...,68.0,69.0,70.0,71.0,74.0,76.0,77.0,328.0,1007.0,1819.0
call_day,,,,,,,,,,,,,,,,,,,,,
1.0,70.771835,63.148148,37.880952,188.246416,105.155844,64.498192,82.203807,81.351141,86.508831,89.177081,...,76.133523,51.225490,74.554924,84.376254,116.861607,44.116189,57.500000,61.216667,51.166667,74.200000
2.0,120.655674,119.250000,50.116667,138.158696,101.018382,173.496635,92.115818,83.962822,87.485855,88.442159,...,84.855263,39.222222,79.604266,96.889706,63.444444,48.623788,83.517857,91.409091,35.000000,90.500000
3.0,156.624542,106.250000,42.522222,92.569501,80.891827,69.309332,94.044978,87.036146,87.543889,89.662111,...,57.517857,28.000000,93.580887,97.905229,80.825397,54.540615,89.727273,94.943850,116.944444,63.018100
4.0,115.614379,253.075000,46.116162,123.230000,123.437500,84.338903,95.332599,86.533552,87.928406,89.834139,...,75.545455,34.294872,91.537760,76.452381,71.848970,58.702747,64.866667,107.288278,77.000000,72.166667
5.0,158.477273,60.535714,48.801852,97.270130,56.145320,71.019501,92.155665,84.259023,87.543499,89.644374,...,60.274436,46.603367,133.534722,465.166667,84.578283,58.868140,53.785714,48.104167,471.000000,97.925000
6.0,184.938889,92.964286,45.670034,88.304642,118.988095,49.515824,98.972566,82.749682,86.343029,88.214886,...,73.979167,37.727273,154.350000,111.666667,106.255952,68.131133,41.917460,58.096405,106.250000,102.708333
7.0,116.259524,219.458333,37.205861,96.403794,87.076923,55.212775,93.802974,84.255500,85.952991,87.856453,...,33.077778,37.070175,107.652083,178.825000,82.791667,52.815975,62.566667,133.166667,56.000000,100.416667
